In [9]:
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import datetime
import matplotlib.pyplot as plt
from keras.models import Input, Model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.initializers import RandomNormal
from keras.callbacks import EarlyStopping
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
def discriminator(images, reuse=False):
    """
        From tutorial: https://www.oreilly.com/learning/generative-adversarial-networks-for-beginners
    """
    if (reuse):
        tf.get_variable_scope().reuse_variables()

    # First convolutional and pool layers
    # This finds 32 different 5 x 5 pixel features
    # Create a 5x5 4d tensor. 5 x 5 x 1 is the pixel patch size (5 x 5) with 1 color chanel
    #                         32 is the output.
    d_w1 = tf.get_variable('d_w1', shape=[5, 5, 1, 32], dtype=tf.float32,
                           initializer=tf.truncated_normal_initializer(stddev=0.01))
    # Bias for each of the 32 outputs.
    d_b1 = tf.get_variable('d_b1', shape=[32], initializer=tf.truncated_normal_initializer(stddev=0.01))
    
    d1 = tf.nn.conv2d(input=images, filter=d_w1, strides=[1, 1, 1, 1], padding='SAME')
    d1 = tf.add(d1, d_b1)
    d1 = tf.nn.relu(d1)
    d1 = tf.nn.max_pool(d1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # Second convolutional and pool layers
    # This finds 64 different 5 x 5 pixel features
    d_w2 = tf.get_variable('d_w2', [5, 5, 32, 64], initializer=tf.truncated_normal_initializer(stddev=0.02))
    d_b2 = tf.get_variable('d_b2', [64], initializer=tf.constant_initializer(0))
    d2 = tf.nn.conv2d(input=d1, filter=d_w2, strides=[1, 1, 1, 1], padding='SAME')
    d2 = tf.add(d2, d_b2)
    d2 = tf.nn.relu(d2)
    d2 = tf.nn.max_pool(d2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # First fully connected layer
    d_w3 = tf.get_variable('d_w3', [7 * 7 * 64, 1024], initializer=tf.truncated_normal_initializer(stddev=0.02))
    d_b3 = tf.get_variable('d_b3', [1024], initializer=tf.constant_initializer(0))
    d3 = tf.reshape(d2, [-1, 7 * 7 * 64])
    d3 = tf.matmul(d3, d_w3)
    d3 = d3 + d_b3
    d3 = tf.nn.relu(d3)

    # Second fully connected layerkeras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)

    d_w4 = tf.get_variable('d_w4', [1024, 1], initializer=tf.truncated_normal_initializer(stddev=0.02))
    d_b4 = tf.get_variable('d_b4', [1], initializer=tf.constant_initializer(0))
    d4 = tf.matmul(d3, d_w4) + d_b4

    # d4 contains unscaled values
    return d4

In [6]:
def keras_discriminator():
    """
        Translation of tutorial into Keras.
    """
    inputs = Input(shape=(28, 28, 1))
    layer = Conv2D(filters=32, kernel_size=(5, 5), activation="relu")(inputs)
    layer = MaxPool2D()(layer)
    layer = Conv2D(filters=64, kernel_size=(5, 5), activation="relu")(layer)
    layer = MaxPool2D()(layer)
    
    layer = Flatten()(layer)
    layer = Dense(units=512, activation="relu")(layer)
    outputs = Dense(units=10, activation="softmax")(layer)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss="categorical_crossentropy", 
              optimizer="adam", 
              metrics=["accuracy"])
    
    return model
    
    

In [12]:
d = keras_discriminator()
d.fit(x=mnist.train.images.reshape((mnist.train.images.shape[0], 28, 28, 1)),
      y=np_utils.to_categorical(mnist.train.labels),
      batch_size=100,
      epochs=10,
      callbacks=[EarlyStopping(patience=2)],
      validation_split=0.2)

Train on 44000 samples, validate on 11000 samples
Epoch 1/10
44000/44000 [==============================] - 56s - loss: 0.1726 - acc: 0.9476 - val_loss: 0.0541 - val_acc: 0.9836
Epoch 2/10
44000/44000 [==============================] - 55s - loss: 0.0463 - acc: 0.9852 - val_loss: 0.0462 - val_acc: 0.9867
Epoch 3/10
44000/44000 [==============================] - 55s - loss: 0.0302 - acc: 0.9907 - val_loss: 0.0393 - val_acc: 0.9878
Epoch 4/10
44000/44000 [==============================] - 55s - loss: 0.0212 - acc: 0.9935 - val_loss: 0.0518 - val_acc: 0.9860
Epoch 5/10
44000/44000 [==============================] - 55s - loss: 0.0167 - acc: 0.9944 - val_loss: 0.0347 - val_acc: 0.9903
Epoch 6/10
44000/44000 [==============================] - 55s - loss: 0.0132 - acc: 0.9957 - val_loss: 0.0348 - val_acc: 0.9906
Epoch 7/10
44000/44000 [==============================] - 55s - loss: 0.0109 - acc: 0.9965 - val_loss: 0.0463 - val_acc: 0.9894
Epoch 8/10
44000/44000 [==============================

In [14]:
err = d.evaluate(x=mnist.test.images.reshape((mnist.test.images.shape[0], 28, 28, 1)),
           y=np_utils.to_categorical(mnist.test.labels))

 9952/10000 [============================>.] - ETA: 0s

In [16]:
print("error rate: ", 1 - err[-1])

error rate:  0.0101


In [33]:
def output_size(w, k, p, s):
    return (w - k + 2 * p) / s + 1

In [52]:
output_size(28, 5, 0, 1)

24.0

In [53]:
output_size(24, 2, 0, 2)

12.0

In [54]:
output_size(12, 5, 0, 1)

8.0

In [97]:
np_utils.to_categorical(mnist.train.labels)

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [89]:
?Model